In [2]:
% reset -f
# !pip install http://download.pytorch.org/whl/cu75/torch-0.2.0.post1-cp27-cp27mu-manylinux1_x86_64.whl
# !pip install torchvision 
# ! pip install cv2

from __future__ import print_function
from __future__ import division
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import MultiLabelBinarizer
# import cv2
# http://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# https://www.bountysource.com/issues/44576966-a-tutorial-on-writing-custom-datasets-samplers-and-using-transforms
# https://medium.com/towards-data-science/my-first-kaggle-competition-9d56d4773607
# https://github.com/sohyongsheng/kaggle-planet-forest

import logging
handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)


## Setting up global variables

In [3]:
DATA_ROOT ='/root/data/amz/'
IMG_PATH = DATA_ROOT + '/train-jpg/'
IMG_EXT = '.jpg'
TRAIN_DATA = DATA_ROOT + '/train_v2.csv'

try:
    from PIL import Image
except ImportError:
    import Image

In [13]:
class GenericImageDataset(Dataset):    

    def __init__(self, csv_path, img_path, img_ext, transform=None):
        
        lgr.info ("CSV path:" + csv_path)
        lgr.info ("IMG path:" + img_path)
        
        assert img_ext in ['.jpg']
        
        tmp_df = pd.read_csv(csv_path, header=None)
                        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df[0]        
#         self.X_train = self.X_train.ix[1:]
        
        self.y_train = self.mlb.fit_transform(tmp_df[1].str.split()).astype(np.float32)         
                
#         lgr.info("DF:\n" + str (self.X_train))
#         lgr.info ("self.y_train:\n" + str(self.y_train))

    def __getitem__(self, index):
        lgr.info ("__getitem__:" + str(index))
        path=self.img_path + self.X_train[index] + self.img_ext
#         lgr.info (" --- get item path:" + path)
        img = Image.open(path)
        img = img.convert('RGB')
        if self.transform is not None: # TypeError: batch must contain tensors, numbers, or lists; 
                                     #found <class 'PIL.Image.Image'>
            img = self.transform(img)
#             print (str (type(img))) # <class 'torch.FloatTensor'>                
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        l=len(self.X_train.index)
        lgr.info ("Lenght:" +str(l))
        return (l)       

    @staticmethod        
    def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

    @staticmethod    
    def flaotTensorToImage(img, mean=0, std=1):
        """convert a tensor to an image"""
        img = np.transpose(img.numpy(), (1, 2, 0))
        img = (img*std+ mean)*255
        img = img.astype(np.uint8)    
        return img    
    
    @staticmethod
    def toTensor(img):
        """convert a numpy array of shape HWC to CHW tensor"""
        img = img.transpose((2, 0, 1)).astype(np.float32)
        tensor = torch.from_numpy(img).float()
        return tensor/255.0


##  transforms.ToTensor(): 

Converts a PIL.Image or numpy.ndarray (H x W x C) in the range [0, 255]  to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].

In [14]:
transformations = transforms.Compose([transforms.ToTensor()])
 
dset_train = GenericImageDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,transformations)

train_loader = DataLoader(dset_train,
                          batch_size=1,
                          shuffle=False,
                          num_workers=1 # 1 for CUDA
                         # pin_memory=True # CUDA only
                         )

import torchvision

# for i, data in enumerate(train_loader, 0):
#             lgr.info('i=%d: '%(i))            
#             images, labels = data
            
#             num = len(images)
#             for n in range(num):
#                 image=images[n]
#                 label=labels[n]
# #                 lgr.info("Label:" + str(label))
#                 plt.imshow (GenericImageDataset.flaotTensorToImage(image))
                

INFO:__main__:CSV path:/root/data/amz//train_v2.csv
INFO:__main__:IMG path:/root/data/amz//train-jpg/


In [15]:
# https://www.kaggle.com/mratsim/starting-kit-for-pytorch-deep-learning/code/notebook

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(2304, 256)
        self.fc2 = nn.Linear(256, 17)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1) # Flatten layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

model = Net() # On CPU
# model = Net().cuda() # On GPU

In [16]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [17]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))                        

In [18]:
for epoch in range(1, 2):
    train(epoch)

INFO:__main__:Lenght:40479
INFO:__main__:__getitem__:0
INFO:__main__:__getitem__:1
INFO:__main__:__getitem__:2


RuntimeError: size mismatch, m1: [1 x 246016], m2: [2304 x 256] at /pytorch/torch/lib/TH/generic/THTensorMath.c:1293